<a href="https://colab.research.google.com/github/ayaanzhaque/DeepRacer2020/blob/master/SSL_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GAN Model

In [0]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pdb

# torch imports
import torch
from torch.utils.data import DataLoader,Dataset
from torch import optim,nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils


class DCGAN_generator(nn.Module):
  """

  Attributes
  ----------
    ngpu : int
      The number of available GPU devices

  """
  def __init__(self, ngpu):
    """Init function

    Parameters
    ----------
      ngpu : int
        The number of available GPU devices

    """
    super(DCGAN_generator, self).__init__()
    self.ngpu = ngpu
        
    # just to test - will soon be args
    nz = 100 # noise dimension
    ngf = 64 # number of features map on the first layer
    nc = 3 # number of channels

    self.main = nn.Sequential(
      # input is Z, going into a convolution
      nn.ConvTranspose2d(     nz, ngf * 4, 4, 1, 0, bias=False),
      nn.BatchNorm2d(ngf * 4),
      nn.ReLU(True),
      # state size. (ngf*8) x 4 x 4
      nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ngf * 2),
      nn.ReLU(True),
      # state size. (ngf*4) x 8 x 8
      nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ngf),
      nn.ReLU(True),
      # state size. (ngf*2) x 16 x 16
      nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
      nn.Tanh()
      # state size. (nc) x 64 x 64
    )

  def forward(self, input):
    """Forward function

    Parameters
    ----------
    input : :py:class:`torch.Tensor`
    
    Returns
    -------
    :py:class:`torch.Tensor`
      the output of the generator (i.e. an image)

    """
    #if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
    #  output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
    #else:
    #  output = self.main(input)
    
    # let's assume that we will never face the case where more than a GPU is used ...
    output = self.main(input)
    return output



class DCGAN_discriminator(nn.Module):
  """ 

  Attributes
  ----------
    ngpu : int
      The number of available GPU devices

  """
  def __init__(self, ngpu):
    """Init function

    Parameters
    ----------
      ngpu : int
        The number of available GPU devices

    """
    super(DCGAN_discriminator, self).__init__()
    self.ngpu = ngpu
        
        
    # just to test - will soon be args
    ndf = 64
    nc = 3
       
    self.main = nn.Sequential(
      nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ndf),
      nn.LeakyReLU(0.2, inplace=True),

      nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ndf * 2),
      nn.LeakyReLU(0.2, inplace=True),
      # state size. (ndf*4) x 8 x 8
      nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ndf * 4),
      nn.LeakyReLU(0.2, inplace=True),
      # state size. (ndf*8) x 4 x 4
      nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False),
      nn.Sigmoid()
    )

  def forward(self, input):
    """Forward function

    Parameters
    ----------
    input : :py:class:`torch.Tensor`
    
    Returns
    -------
    :py:class:`torch.Tensor`
      the output of the generator (i.e. an image)

    """
    #if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
    #  output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
    #else:
    #  output = self.main(input)
    
    # let's assume that we will never face the case where more than a GPU is used ...
    output = self.main(input)

    return output.view(-1, 1).squeeze(1)

transform = transforms.Compose(
    [transforms.ToTensor()])

batch_size = 64
trainset = datasets.SVHN("/content", split='train', download = True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2,)

testset = datasets.SVHN("/content", split='test', download = True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# define device 
device = torch.device("cuda:0")

# data for plotting purposes
generatorLosses = []
discriminatorLosses = []

#training starts

epochs = 25

input_size = 32

real_label = 1
fake_label = 0


# models
netG = DCGAN_generator(1)
netD = DCGAN_discriminator(1)

netG.to(device)
netD.to(device)

print(netG)

# optimizers 
optD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999)) 
optG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999)) 

input_length = int(math.log(128, 2))

loss = nn.BCELoss()

for epoch in range(epochs):

  for i, data in enumerate(trainloader, 0):
    
    dataiter = iter(trainloader)
    inputs, labels = dataiter.next()
    inputs, labels = inputs.to(device), labels.to(device)
    tmpBatchSize = len(labels)  

    # create label arrays 
    true_label = torch.ones(tmpBatchSize, 1, device=device)
    fake_label = torch.zeros(tmpBatchSize, 1, device=device)
    # print(inputs)
    # print(labels)

    # generate fake images // im struggling here as well
    r = torch.randn(tmpBatchSize, 100, 1, 1, device=device) #not sure if this is correct but it isnt giving errors
    # print(r)
    fakeImageBatch = netG(r)
    # print(fakeImageBatch)

    # # visualize the fake image 
    # plt.subplot(1,2,2)
    # plt.axis("off")
    # plt.title("Fake Images")
    # plt.imshow(np.transpose(vutils.make_grid(fakeImageBatch, padding=2, normalize=True)))
    # plt.show()

    real_cpu = data[0].to(device)
    batch_size = real_cpu.size(0)
    # print(batch_size)

    # train generator on real images
    # predictionsReal = netD(real_cpu).view(-1)
    predictionsReal = netD(inputs)
    lossDiscriminator = loss(predictionsReal, true_label) #labels = 1
    lossDiscriminator.backward(retain_graph = True)

    # train generator on fake images
    predictionsFake = netD(fakeImageBatch)
    lossFake = loss(predictionsFake, fake_label)  #labels = 0
    lossFake.backward(retain_graph= True)
    optD.step() # update discriminator parameters    

    # train generator 
    optG.zero_grad()
    predictionsFake = netD(fakeImageBatch)
    # batch_size = 8192
    # true_label = torch.full((batch_size,), real_label, device=device)
    lossGenerator = loss(predictionsFake, true_label) #labels = 1
    lossGenerator.backward(retain_graph = True)
    optG.step()

    # reset the gradients
    optD.zero_grad()
    optG.zero_grad()

    # save losses for graphing
    generatorLosses.append(lossGenerator.item())
    discriminatorLosses.append(lossDiscriminator.item())

    # # save generated images 
    if(i % 100 == 0):
       gridOfFakeImages = torchvision.utils.make_grid(fakeImageBatch.cpu())
       torchvision.utils.save_image(gridOfFakeImages, "/content/gridOfFakeImages/" + str(epoch) + '_' + str(i) + '.png')

  print("Epoch " + str(epoch) + "Complete")
  print("Generator Loss: " + str(lossGenerator))
  print("Discriminator Loss: " + str(lossDiscriminator))

def validate():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

#save models
torch.save(netG, "netG.h5")
torch.save(netD, "netD.h5")

# plot losses
plt.figure(figsize=(10,5))
plt.title("Loss of Models")
plt.plot(generatorLosses,label="Generator")
plt.plot(discriminatorLosses,label="Discriminator")
plt.xlabel("Batches")
plt.ylabel("Loss")
plt.legend()
plt.show()

Tasks
1. Train a Working GAN (done)
2. Train a general classifer (Train the same classifer at different dataset sizes, plot its datasize vs accuracy)
3. Train a classifier that uses the smaller dataset sizes + the generated images 
4. Export accuracy vs dataset size into a different file
5. plot the data from the file, see the difference in accuracy

# Classifier

Classifier with different dataset sizes and generated images

In [0]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pdb

# torch imports
import torch
from torch.utils.data import DataLoader,Dataset
from torch import optim,nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data
import torchvision.utils as vutils

# model
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16
        self.embDim = 128 * block.expansion
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 128, num_blocks[3], stride=2)
        self.linear = nn.Linear(128 * block.expansion, num_classes)
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        emb = out.view(out.size(0), -1)
        out = self.linear(emb)
        return out, emb
    def get_embedding_dim(self):
        return self.embDim

def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

#data preprocessing

file = open("modelData.txt", "w")

transform = transforms.Compose(
    [transforms.ToTensor()])

generatedImagesRoot = "/content/drive/My Drive/Colab Notebooks/gridOfFakeImages"

batch_size = 64
trainset = datasets.SVHN("/content", split='train', download = True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2,)

print(trainloader)

testset = datasets.SVHN("/content", split='test', download = True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

dataSizeConstant = 0.1
subset = np.random.permutation([i for i in range(len(trainset))])
subTrain = subset[:int(len(trainset) * dataSizeConstant)]

print(subset)
print(subTrain)

subTrainSet = datasets.SVHN("/content", split = "test", download = True, transform = transform)
subTrainLoader = DataLoader(trainset, batch_size = batch_size, shuffle= False, num_workers= 2, sampler = torch.utils.data.SubsetRandomSampler(subTrain))

# define device 
device = torch.device("cuda:0")

# data for plotting purposes
modelLoss = []

# model
net = ResNet(BasicBlock, [2,2,2,2])
net.to(device)
opt = optim.Adam(net.parameters(), lr=0.0001, betas=(0.5, 0.999)) 
criterion = nn.CrossEntropyLoss().cuda()

#training starts

epochs = 25

# datasetLoader = subTrainLoader
def train(datasetLoader):
  # file.write("Datasize: %d %%" % (dataSizeConstant))
  text = ("Datasize: " + str(dataSizeConstant) + "/n")
  file.write(text)

  net.train()
  for epoch in range(epochs):

    running_loss = 0.0
    for i, data in enumerate(datasetLoader, 0):
      dataiter = iter(datasetLoader)
      inputs, labels = dataiter.next()
      inputs, labels = inputs.to(device), labels.to(device)

      # print(inputs)
      # print(labels)

      opt.zero_grad()

      outputs = net(inputs)
      # print(outputs)
      pdb.set_trace()
      modelLoss = criterion(outputs, labels) # error line
      modelLoss.backward()
      opt.step()

      print(modelLoss)

      net.eval()
      # accuracy
      _, predicted = torch.max(outputs.data, 1)
      total_train += mask.size(0)
      correct_train += predicted.eq(mask.data).sum().item()
      train_accuracy = 100 * correct_train / total_train
      #avg_accuracy = train_accuracy / len(train_loader)                                     
      
      print('Epoch {}, train Loss: {:.3f}'.format(epoch, loss.item()), "Training Accuracy: %d %%" % (train_accuracy))

      # # save generated images 
      if(i % 100 == 0):
        text = ("Train Accuracy: " + str(train_accuracy))
        file.write(text)



    print("Epoch " + str(epoch) + "Complete")
    print("Loss: " + str(loss))

# validation 
def validate():
  net.eval()
  correct = 0
  total = 0
  with torch.no_grad():
      for data in testloader:
          images, labels = data
          outputs = net(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  accuracy = (correct / total) * 100 

  print('Accuracy of the network on the 10000 test images: %d %%' % (
      100 * correct / total))

  text = ("Datasize: " + str(dataSizeConstant) + "/n")
  file.write(text)
  text = ("Train Accuracy: " + str(accuracy))
  file.write(text)

train(subTrainLoader)

file.close() 

Using downloaded and verified file: /content/train_32x32.mat
Using downloaded and verified file: /content/test_32x32.mat
[10509 19450  5777 ... 37890 18110 19355]
[10509 19450  5777 ... 27735 63159  9328]
Using downloaded and verified file: /content/test_32x32.mat
> <ipython-input-2-16112d959262>(148)train()
-> modelLoss = criterion(outputs, labels) # error line
(Pdb) type(outputs)
<class 'tuple'>
(Pdb) type(labels)
<class 'torch.Tensor'>
(Pdb) modelLoss = criterion((outputs, 10), labels)
*** AttributeError: 'tuple' object has no attribute 'log_softmax'
(Pdb) input = torch.randn(3, 5, requires_grad=True)
(Pdb) modelLoss = criterion(input, labels)
*** ValueError: Expected input batch_size (3) to match target batch_size (64).
(Pdb) input = torch.randn(64, 5, requires_grad = True)
(Pdb) modelLoss = criterion(input, labels)
*** RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _thnn_nll_loss_forward
(Pdb) input = torch.randn(3, 5, r